This module contains the implementation of the PPO algorithm.
Ci basiamo sullo pseudocodice presente sul sito di OpenAI per la realizzazione del ppo.
https://spinningup.openai.com/en/latest/algorithms/ppo.html#id7
Utilizzando un Actor-Critic Method.
Ciò suddivide l'implementazione in 8 passi principali:
1. Inizializzazione dell'ambiente con policy parameters theta_0, e l'inizial value function parameters w_0.
2. Ciclare per k iterazioni
3. Raccogliere un set di traiettorie D_k = {τ_i} con una policy pi_k = pi(theta_k)
4. Calcolare i reward-to-go R_t
5. Calcolare gli advantage estimates A_t basandoci sulla value function V_{w_k}
6. Aggiornare la policy massimizzando la PPO-Clip objective (Gradient ascent con adam) . Non scriverò la formula che è complessa
7. Aggiornare la value function minimizzando la MSE tra V_{w_k} e R_t (Gradient descent con adam)
8. Fine ciclo.

Implementiamo tutti i passi nella funzione learn.

In [1]:
import warnings
warnings.filterwarnings('ignore') #ignora warnings
#Check if colab is used:
from rete import ReteNeurale
import tensorflow as tf
import tensorflow_probability as tfp
import gym
import numpy as np
from tensorflow import keras
import pandas as pd
import matplotlib.pyplot as plt
import glfw
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False
  print("Not running on CoLab")
  #print list of GPUs
  #tf. config. list_physical_devices('GPU')
  print("Devices: ", tf.config.list_physical_devices())
  print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
if IN_COLAB:
  !pip install procgen
  !pip install tensorflow_probability
  !pip install numpy


2024-12-27 19:32:38.435201: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1735324358.450842  253168 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1735324358.455292  253168 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-27 19:32:38.471065: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Not running on CoLab
Devices:  [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Num GPUs Available:  1


In [2]:
#Tutta questa parte è relativa alla visualizzazione del gioco tramite salvataggio degli stati di un episode in una specifica cartella.
#Tutto questo viene fatto perchè non si riesce a visualizzare procgen in live per problemi con glfw e openGL su Ubuntu.
from moviepy import ImageSequenceClip
from IPython.display import Video
import os
from pyvirtualdisplay.smartdisplay import SmartDisplay
display = SmartDisplay(visible=0, size=(1920,1080),fbdir='/tmp')
display.start()
glfw.init()
available_fbconfigs = glfw.get_video_modes(glfw.get_primary_monitor())
os.environ['PYOPENGL_PLATFORM'] = 'osmesa'




In [ ]:
class PPO:
    def __init__(self,env,gameName,totalSteps=2000000):
        self.env=env
        self.gameName=gameName
        self.nAzioni=env.action_space.n
        self.nStati=env.observation_space.shape
        self.listaAzioni=[i for i in range(self.nAzioni)]
        self.nTimestampsPerBatch=8192
        #self.nTimestampsPerBatch=4096
        self.stepsPerEpisode=512
        self.nTotalTimestamps=totalSteps
        self.episodesPerBatch=10
        self.nEpoche=300
        self.gamma=0.99
        self.epsilon=0.2
        self.learningRate=1e-4
        self.policyNN=ReteNeurale(self.nStati,self.nAzioni) #Actor
        self.policy_optimizer=keras.optimizers.Adam(learning_rate=self.learningRate, clipnorm=1.0)
        self.policyNN.compile(optimizer=self.policy_optimizer)
        self.entropyCoefficient=0.01 #Per invogliare l'esplorazione un po di più.
        self.lambdaGAE=0.95
        self.updateLearningRateEveryTimesteps=10000000 #Aggiorna il learning rate ogni x steps dove x è il valore della variabile
        self.csvPath="./rewards/"+self.gameName+"_rewards.csv"
        self.offsetCsv=0 #Usato per capire da quale riga iniziare a scrivere i rewards. Per non sovrascrivere i vecchi rewards.
        self.batchSize=512
        #Creo il file csv per salvare i rewards
        if not os.path.isfile(self.csvPath):
            data = {"Epoch": [], "Average reward": [], "Min reward": [], "Max reward": []}
            df=pd.DataFrame(data,columns=["Epoch","Average reward","Min reward","Max reward"])
            df.to_csv(self.csvPath,index=False,header=True)
        else:
            df=pd.read_csv(self.csvPath)
            self.offsetCsv=len(df.index)
        

    def learn(self):
        #passo 2 ciclare per k iterazioni.
        stepsTot=0
        iterazioniTot=0
        while stepsTot<self.nTotalTimestamps:
            print("Step totali eseguiti: {}".format(stepsTot)," Step totali rimasti:",self.nTotalTimestamps-stepsTot)
            self.updateLearningRate(stepsTot) 
            states, actions, rewards_to_go, log_probs, dones,len_ep =self.collect_trajectories()
            stepsTot+=np.sum(len_ep)
            iterazioniTot+=1
            num_samples=np.sum(len_ep)
            print("NUM SAMPLES:",num_samples)

            samplesInPiu=num_samples%self.batchSize
            batch_size=self.batchSize+samplesInPiu

            i=0
            while i <num_samples:
                batch_states=states[i:i+batch_size]
                batch_actions=actions[i:i+batch_size]
                batch_rewards_to_go=rewards_to_go[i:i+batch_size]
                batch_log_probs=log_probs[i:i+batch_size]
                batch_dones=dones[i:i+batch_size]

                v,latest_log_probs,_=self.evaluate(batch_states,batch_actions)
                #advantage=self.calcAdvantages(batch_rewards_to_go,V)
                advantage, targets =self.calcGaeAndTargets(batch_rewards_to_go, v, batch_dones)
                
                with tf.GradientTape() as tape:
                    _,latest_log_probs,probs=self.evaluate(batch_states,batch_actions)
                    policy_loss = self.getPolicyLoss(batch_log_probs,latest_log_probs,advantage)
                    #MSE valueLoss
                    value_loss = tf.reduce_mean(tf.square(targets - v))
                    
                    #Aggiungo entropia alla loss per incentivare l'esplorazione
                    entropy = -tf.reduce_mean(probs * tf.math.log(probs + 1e-10))
                    total_loss=policy_loss+ value_loss*0.5 - entropy*self.entropyCoefficient
                gradientsPolicy = tape.gradient(total_loss, self.policyNN.trainable_variables)
                self.policy_optimizer.apply_gradients(zip(gradientsPolicy, self.policyNN.trainable_variables))
                i+=batch_size
                batch_size=self.batchSize
                print("EPOCA:",iterazioniTot," TOTAL LOSS:",total_loss.numpy()," POLICY LOSS:",policy_loss.numpy()," VALUE LOSS:",value_loss.numpy()," ENTROPY:",entropy.numpy())
            self.evaluate_policy(epoch=iterazioniTot)
            if iterazioniTot%10==0:
                self.saveModel("ppo_"+self.gameName+".weights.h5")
                



    def evaluate_policy(self, episodes=10,epoch=0):
        total_rewards = []
        for i in range(episodes):
            frames=[]

            state = self.env.reset()
            frames.append(state)
            done = False
            cumulative_reward = 0
            while not done:
                state_tensor = tf.convert_to_tensor(state, dtype=tf.float32)
                state_tensor = tf.expand_dims(state_tensor, axis=0)
                probs, _ = self.policyNN(state_tensor)
                action = np.argmax(probs.numpy())
                state, reward, done, _ =self.env.step(action)
                frames.append(state)

                cumulative_reward += reward
            total_rewards.append(cumulative_reward)
            self.saveClip(frames,i)
            
            print("Episode reward:", cumulative_reward)
        print(f"Average Reward: {np.mean(total_rewards):.2f}")
        self.saveReward(np.mean(total_rewards),np.min(total_rewards),np.max(total_rewards),epoch,"./rewards/"+self.gameName+"_rewards.csv")

    def collect_trajectories(self):
        #Passo 3 --> Raccogliere un set di traiettorie D_k = {τ_i} con una policy pi_k = pi(theta_k)
        #Dobbiamo raccogliere un set di traiettorie e per fare ciò dobbiamo raccogliere: stati, azioni, rewards, rewards to go, log_prob delle azioni.
        batch={
            'states':[],
            'actions':[],
            'rewards':[],
            'rewards_to_go':[],
            'log_probs':[],
            'done':[],
            'lengths':[]
        }

        t = 0 # Keeps track of how many timesteps we've run so far this batch
        nEpisodes=0
        while t < self.nTimestampsPerBatch:
            rewardPerEpisode=[]
            stato = self.env.reset()
            done = False
            frames=[]
            for i in range(self.stepsPerEpisode):
                t+=1
                batch['states'].append(stato)
                azione,log_prob=self.getAction(stato)
                batch['actions'].append(azione)
                batch['log_probs'].append(log_prob)
                stato, reward, done ,_= self.env.step(azione)  #al posto di _ ci sarebbe info ma non ci serve
                rewardPerEpisode.append(reward)
                frames.append(stato)
                batch['done'].append(done)
                if done :
                    break #Ha raggiunto il termine dell'episodio.
            batch['rewards'].append(rewardPerEpisode)
            batch['lengths'].append(i+1)
            nEpisodes+=1
            #self.saveClip(frames,nEpisodes)
            frames=[]
        #Calcoliamo i rewards to go --> PASSO 4
        batch['rewards_to_go']=self.calcRTG(batch['rewards'])
        batch_statiTensor=tf.convert_to_tensor(batch['states'],dtype=tf.uint8)
        batch_azioniTensor=tf.convert_to_tensor(batch['actions'],dtype=tf.int32)
        batch_rewards_to_goTensor=tf.convert_to_tensor(batch['rewards_to_go'],dtype=tf.float32)
        batch_log_probsTensor=tf.convert_to_tensor(batch['log_probs'],dtype=tf.float32)
        batch_len=tf.convert_to_tensor(batch['lengths'],dtype=tf.int32)
        batch_dones=tf.convert_to_tensor(batch['done'],dtype=tf.bool)


        return batch_statiTensor, batch_azioniTensor,batch_rewards_to_goTensor,batch_log_probsTensor, batch_dones,batch_len

    def getAction(self,stato):
        stato=tf.convert_to_tensor(np.expand_dims(stato, axis=0) ,dtype=tf.float32)# Diventa (1, 64, 64, 3)
        azione_pred,_=self.policyNN(stato)
        #Somma probabilità
        dist=tfp.distributions.Categorical(probs=tf.squeeze(azione_pred))
        azionePresa=dist.sample()
        log_prob=dist.log_prob(azionePresa)
        return azionePresa, tf.stop_gradient(log_prob)

    def calcRTG(self,rewards):
        #Prendo la formula per calcolare i rewards to go e richiede i cumulative rewards e un fattore di sconto.
        rtg=[]
        for episode_reward in reversed(rewards):
            cumulative_reward=0
            totalRewardPerEpisode=0
            for single_reward in reversed(episode_reward):
                cumulative_reward=single_reward+cumulative_reward*self.gamma
                totalRewardPerEpisode+=single_reward
                rtg.append(cumulative_reward)
            print("Total reward per episode RTG:",totalRewardPerEpisode)
        return tf.convert_to_tensor(rtg,dtype=tf.float32)

   
    def calcGaeAndTargets(self,rewards,values,dones):
        advantages = []
        targets = []
        advantage = 0
        try:
            tf.debugging.check_numerics(rewards, "Ricompense non valide")
            tf.debugging.check_numerics(values, "Valori non validi")
            tf.debugging.assert_type(dones, tf.bool, "La variabile dones deve essere booleana.")
            
        except:
            print("Errore: ",rewards , values, dones)
        for t in reversed(range(len(rewards))):

            #Se una delle variabili è solo un valore, allora non posso fare slicing e devo fare un controllo.
            if t+1<len(rewards):
                delta=rewards[t]+ (1-dones[t])*self.gamma*values[t+1]-values[t]
            else:
                delta=rewards[t]-values[t]
            advantage=delta+self.gamma*self.lambdaGAE*(1-dones[t])*advantage
            advantages.insert(0,advantage)
            targets.insert(0, advantage + values[t])
        return tf.convert_to_tensor(advantages, dtype=tf.float32), tf.convert_to_tensor(targets, dtype=tf.float32)


    def getPolicyLoss(self,log_probs_old, log_probs_new, advantages):
        advantages = tf.stop_gradient(advantages)
        policy_ratio = tf.exp(log_probs_new-log_probs_old)
        surrogated_loss_1 = policy_ratio * advantages
        clipped_policy_ratio=tf.clip_by_value(policy_ratio, clip_value_min=1.0-self.epsilon, clip_value_max=1.0+self.epsilon)
        surrogated_loss_2 = clipped_policy_ratio * advantages
        clip_loss=tf.minimum(surrogated_loss_1,surrogated_loss_2)
        return -tf.reduce_mean(clip_loss)

    def evaluate(self, batch_states,batch_actions):
        batch_states=tf.cast(batch_states, tf.float32)
        mean,v=self.policyNN(batch_states)
        mean = tf.clip_by_value(mean, 1e-10, 1.0)  # Evita valori molto bassi
        mean /= tf.reduce_sum(mean, axis=-1, keepdims=True)  # Normalizza
        v= tf.squeeze(v)
        dist=tfp.distributions.Categorical(probs=mean)
        log_probs=dist.log_prob(batch_actions)
        return v, log_probs, mean

    def loadModel(self, path):
        if path is "":
            return
        self.policyNN.build(self.nStati)
        try:
            #Check if weights contains Nan or Inf
            self.policyNN.load_weights(path)
            for var in self.policyNN.trainable_variables:
                tf.debugging.check_numerics(var, "LOAD Contiene NAN o INF")

        except:
            print("Errore nel caricamento del modello")


    def saveModel(self, path):
        #Controllo che la cartella esista e che non ci siano NaN nei pesi
        if not os.path.exists("weights"):
            os.makedirs("weights")
        try:
            for var in self.policyNN.trainable_variables:
                tf.debugging.check_numerics(var, "SAVE Contiene NAN o INF")

        except: 
            print("Errore: i pesi contengono NaN o Inf. Non verrà salvato")
            return
        
        self.policyNN.save_weights(path)
    
    def updateLearningRate(self, epoch):
      if epoch % self.updateLearningRateEveryTimesteps == 0 and epoch > 0:
        self.learningRate *= 0.9  # Riduci il learning rate del 10%
        self.policy_optimizer.learning_rate = self.learningRate #Aggiorno solo dentro l'if che tanto è uguale per tutte le altre volte.   

    def saveReward(self,reward,minReward,maxReward,epoch,path):
        #Devo controllare se c'è davvero il file o meno. In caso affermativo conto quante righe ci sono.Da li ci sarà un offset così da incrementare correttamente
        epoch+=self.offsetCsv        
        data = {"Epoch": [epoch], "Average reward": [reward], "Min reward": [minReward], "Max reward": [maxReward]}
        df=pd.DataFrame(data,columns=["Epoch","Average reward","Min reward","Max reward"])
        df.to_csv(path,mode='a',index=False,header=False)

    def showGraph(self):
        rewards=pd.read_csv("./rewards/"+self.gameName+"_rewards.csv")
        rewards.plot(x='Epoch',y='Average reward',kind='line',title="Average reward per epoch")
        plt.show()
    
    def saveClip(self,frames,i):
        clip = ImageSequenceClip(list(frames), fps=15)
        nameVideo="./clip/"+self.gameName+"/"+self.gameName+"_video"+str(i)+".mp4"
        clip.write_videofile(nameVideo, fps=15,logger=None)
        Video(nameVideo)


In [4]:
# Configurazione ed esecuzione
#Lista di giochi a disposizione di Procgen:
""" 
    bigfish, bossfight, caveflyer, chaser, climber
    coinrun, dodgeball, fruitbot, heist, jumper
    leaper, maze, miner, ninja, plumber, starpilot
"""
seed=42
gameName="starpilot" #Scelto starpilot perchè è un gioco che ha episode corti, quindi allenamenti più rapidi.
env = gym.make('procgen:procgen-'+gameName+'-v0',distribution_mode='easy',start_level=seed,rand_seed=seed, num_levels=100, use_backgrounds=False)

#Creo l'oggetto PPO
ppo_model=PPO(env,gameName)

#load model weights if available 
ppo_model.loadModel("./weights/ppo_"+gameName+".weights.h5")
ppo_model.learn()

#save model weights
ppo_model.saveModel("./weights/ppo_"+gameName+".weights.h5")

ppo_model.showGraph()


I0000 00:00:1735324362.590036  253168 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 2784 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1650, pci bus id: 0000:01:00.0, compute capability: 7.5


Step totali eseguiti: 0  Step totali rimasti: 2000000


I0000 00:00:1735324363.221143  253168 cuda_dnn.cc:529] Loaded cuDNN version 90300
/home/zakaria/anaconda3/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:219: DeprecationWarning: WARN: Core environment is written in old step API which returns one bool instead of two. It is recommended to rewrite the environment with new step API. 
  logger.deprecation(
/home/zakaria/anaconda3/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:225: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(done, (bool, np.bool8)):


Total reward per episode RTG: 0.0
Total reward per episode RTG: 3.0
Total reward per episode RTG: 0.0
Total reward per episode RTG: 2.0
Total reward per episode RTG: 0.0
Total reward per episode RTG: 0.0
Total reward per episode RTG: 0.0
Total reward per episode RTG: 0.0
Total reward per episode RTG: 0.0
Total reward per episode RTG: 0.0
Total reward per episode RTG: 0.0
Total reward per episode RTG: 0.0
Total reward per episode RTG: 1.0
Total reward per episode RTG: 1.0
Total reward per episode RTG: 10.0
Total reward per episode RTG: 5.0
Total reward per episode RTG: 3.0
Total reward per episode RTG: 1.0
Total reward per episode RTG: 2.0
Total reward per episode RTG: 2.0
Total reward per episode RTG: 1.0
Total reward per episode RTG: 1.0
Total reward per episode RTG: 4.0
Total reward per episode RTG: 0.0
Total reward per episode RTG: 6.0
Total reward per episode RTG: 0.0
Total reward per episode RTG: 0.0
Total reward per episode RTG: 0.0
Total reward per episode RTG: 0.0
Total reward 

UnboundLocalError: local variable 'batch_size' referenced before assignment